In [74]:
%load_ext autoreload
%autoreload 1
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot
import cv2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
# custom imports
%aimport database_utils 
%aimport image_utils
%aimport models 

In [76]:
CONNECTION_STRING = 'connect_local.txt'
NUM_TEST_SAMPLES = 500
NUM_CLASSES = 11

In [77]:
m = []
with open('misclassified.csv') as f: 
    for row in f.readlines(): 
        items = row.strip('\n').split(',')
        m.append(items[0])

In [78]:
conn = database_utils.makeDatabaseConnection(CONNECTION_STRING)

In [79]:
record = getMisclassifiedImage(conn, m)
data = [list(d) for d in zip(*record)]
labels, house_ids = data[1], data[2]
image_arr = image_utils.makeImageArrayFromDataBase(data[0])

In [80]:
gen_hflip = ImageDataGenerator(horizontal_flip=True)
gen_vflip = ImageDataGenerator(vertical_flip=True)
gen_zoom = ImageDataGenerator(zoom_range=[0.5,1.0])
gen_bright = ImageDataGenerator(brightness_range=[0.2,1.0])

In [84]:
generateImages(gen_hflip,image_arr,house_ids,labels,conn, 'horizontal')
generateImages(gen_vflip,image_arr,house_ids,labels,conn, 'vertical')
generateImages(gen_zoom,image_arr,house_ids,labels,conn, 'zoom')
generateImages(gen_bright,image_arr,house_ids,labels,conn, 'brightness')

In [81]:
def insertAugmentedImageIntoDatabase(conn, house_id, image, label): 
    curr = conn.cursor()
    data_aug_query = """ INSERT INTO images (house_id, url_path, label, url) VALUES (%s,%s,%s,%s)"""
    record_data_aug = (house_id, image, label, "generated")
    curr.execute(data_aug_query, record_data_aug)
    conn.commit()

In [82]:
def getMisclassifiedImage(conn, image_ids): 
    curr = conn.cursor()
    image_ids = tuple(image_ids)
    mis_sample_query = f"SELECT url_path,label,house_id FROM images where image_id IN {image_ids}"
    curr.execute(mis_sample_query)
    results = curr.fetchall()
    return results

In [83]:
def generateImages(generator, data, house_ids, labels, conn, typeGen): 
    it = generator.flow(data, batch_size=len(data))
    batch = it.next()
    for b, h, l in zip(batch, house_ids, labels):
        image = b.astype('uint8')
        path = 'data/train/' + typeGen + str(h) + '.jpg'
        cv2.imwrite(path, image)
        insertAugmentedImageIntoDatabase(conn, h, path, l)